In [1]:
from DB import DataFrameHandler
from DB import AIModelHandler
import pandas as pd
import pickle, time
pd.options.mode.chained_assignment = None

with open('ohlcv.sav', 'rb') as file:
    ohlcv = file.read()
ohlcv = pickle.loads(ohlcv)

def calcStochasticK(df, n=14):
    Numerator = df - df.rolling(window=n).min()
    Denominator = df.rolling(window=n).max() - df.rolling(window=n).min()
    result = (Numerator / Denominator * 100).round()
    return result

def makeStatisticsDF(df):
    tmp = df.sort_values('S_DATE').reset_index(drop=True)
    tmp.rename(columns={'S_DATE' : 'WORK_DT'}, inplace=True)
    tmp['WORK_DT'] = tmp['WORK_DT'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d')) 
    tmp.drop(columns=['OPEN','HIGH','LOW',"AMOUNT",'UPDOWN'], inplace=True)
    tmp['PRICE_CHANGE'] = tmp['CLOSE'].diff().fillna(0)
    tmp['MOMENTUM'] = (tmp['CLOSE'] - tmp['CLOSE'].shift(10)).fillna(0)
    tmp['OBV'] = ((tmp['PRICE_CHANGE'] != 0) * ((tmp['PRICE_CHANGE'] > 0) * 2 - 1) * tmp['VOLUME']).cumsum()
    tmp['STOCHASTIC_K'] = calcStochasticK(tmp['CLOSE'])
    tmp['STOCHASTIC_D'] = tmp['STOCHASTIC_K'].rolling(window=3).mean()
    tmp['GAIN'] = tmp['PRICE_CHANGE'].apply(lambda x: max(x, 0))
    tmp['LOSS'] = tmp['PRICE_CHANGE'].apply(lambda x: -min(x, 0))
    tmp['EMA_GAIN'] = tmp['GAIN'].ewm(span=10, min_periods=10).mean()
    tmp['EMA_LOSS'] = tmp['LOSS'].ewm(span=10, min_periods=10).mean()
    tmp['RS'] = tmp['EMA_GAIN'] / tmp['EMA_LOSS']
    tmp['RSI'] = 100 - (100 / (1 + tmp['RS']))
    tmp.drop(columns=['CLOSE', 'VOLUME', 'PRICE_CHANGE', 'GAIN', 'LOSS', 'EMA_GAIN', 'EMA_LOSS','RS'], inplace=True)
    tmp = tmp.round().dropna().reset_index(drop=True)
    return tmp

for df in ohlcv:
    test = makeStatisticsDF(df)
    DataFrameHandler.insertItems('TMP_STATISTICS', test)
AIModelHandler.mergeTmpIntoMain('STATISTICS')

MERGE INTO STATISTICS USING TMP_STATISTICS ON (STATISTICS.ISIN = TMP_STATISTICS.ISIN and STATISTICS.WORK_DT = TMP_STATISTICS.WORK_DT) WHEN MATCHED THEN UPDATE SET STATISTICS.RSI = TMP_STATISTICS.RSI, STATISTICS.OBV = TMP_STATISTICS.OBV, STATISTICS.STOCHASTIC_K = TMP_STATISTICS.STOCHASTIC_K, STATISTICS.STOCHASTIC_D = TMP_STATISTICS.STOCHASTIC_D, STATISTICS.MOMENTUM = TMP_STATISTICS.MOMENTUM WHEN NOT MATCHED THEN INSERT (ISIN, WORK_DT, RSI, OBV, STOCHASTIC_K, STOCHASTIC_D, MOMENTUM) VALUES (TMP_STATISTICS.ISIN, TMP_STATISTICS.WORK_DT, TMP_STATISTICS.RSI, TMP_STATISTICS.OBV, TMP_STATISTICS.STOCHASTIC_K, TMP_STATISTICS.STOCHASTIC_D, TMP_STATISTICS.MOMENTUM)
